In [1]:
# default_exp review

# Review

> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#export
from literature_review.article import make_articles
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import re
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import json


In [5]:
import time
class Article_Displayer():
    def __init__(self, articles, checks = [], current = 0, search = None, only_unreviewed = True):
        
        self.current = 0 # current article to display
        def is_reviewed(a):
            if 'reviewed' in a.annotations.keys():
                if a.annotations['reviewed']:
                    return True
            return False
            
        if only_unreviewed:
            articles = [a for a in articles if not is_reviewed(a)]
        self.articles = articles
        self.article = self.articles[current]
        self.checks = ['reviewed'] + checks 
        self.search = search
        self.create_ui()
        self.update_ui()
        
        
    def next_article(self, b):        
        self.change_article(1)
        
    def last_article(self, b):        
        self.change_article(-1)
        
    def change_article(self, increment):
        self.abstract.value = ""
        self.search_results.value = ""
        with self.output:
            self.persist_data()
            self.current += increment
            self.article = self.articles[self.current]
            self.update_ui()
            
    def store_input(self):
        for check in self.check_boxes:
            self.article.annotations[check.description] = check.value
        self.article.annotations['note'] = self.note.value
        
    def persist_data(self):
        self.store_input()
        with open(self.article.path, 'w') as fp:
            json.dump(dict(self.article), fp)
            
    def update_ui(self):
        with self.output:
            self.title.value = "<h3>%s</h3><i>%s<i>."%(self.articles[self.current].title,self.articles[self.current].path)
            self.abstract.value = "<h3>Abstract:</h3>"+self.articles[self.current].abstract
            
        
            self.count_display.value = "%d of %d articles reviewed in this session."%(self.current, len(self.articles))
            for checkbox in self.check_boxes:
                if checkbox.description in self.article.annotations.keys():
                    checkbox.value = self.article.annotations[checkbox.description]
                else:
                    checkbox.value = False
                if checkbox.description == 'reviewed':
                    checkbox.value = True
            if "note" in self.article.annotations.keys():
                self.note.value = self.article.annotations["note"]
            else:
                self.note.value = ""
            if self.search:
                self.search_results.value = "<h3>Search results:</h3>"+self.extract_sentences(self.search)
            
        
    def extract_sentences(self, search_term):
        punkt_param = PunktParameters()
        punkt_param.abbrev_types = set(['al'])
        sentence_splitter = PunktSentenceTokenizer(punkt_param)
        result = ""
        for section in self.article.sections:
            matches = re.findall(search_term, section['text']) 
            if len(matches):
                result += "<h4>%s</h4>"%section['heading']
                matches = list(set(matches))
                sentences = sentence_splitter.tokenize(section['text'])
                matched_sentences = []
                for i, s in enumerate(sentences):
                    for match in matches:
                        if match in s:
                            s = s.replace(match, "<b>%s</b>"%match)
                            matched_sentences.append(sentences[i-1]+' '+s)
                for ms in matched_sentences:
                    result += "<p>%s</p>"%ms
        return result        
    
    def create_ui(self):
        a = self.article
        self.output = widgets.Output()
        self.next_button = widgets.Button(description="Next")
        self.last_button = widgets.Button(description="Last")

        self.next_button.on_click(self.next_article)
        self.last_button.on_click(self.last_article)


        
        self.button_ui = widgets.HBox([self.last_button, self.next_button])
        self.check_boxes = []
        for check in self.checks:
            self.check_boxes.append(widgets.Checkbox(description=check, value= False))
            
        self.note = widgets.Textarea(
            value='',
            placeholder='Add notes here',
            description='String:',
            disabled=False
        )
        
        self.count_display = widgets.HTML(value="count")
        
        # Making the text UI
        self.title = widgets.HTML(value="Title")
        self.abstract = widgets.HTML(value="Abstract")
        self.search_results = widgets.HTML(value="")
        self.text_ui = widgets.VBox([self.title, self.abstract, self.search_results], layout=Layout(width='60%'))
        
        # Making the action ui
        self.action_ui = widgets.VBox([self.button_ui, self.count_display] + self.check_boxes + [self.note, self.output], layout=Layout(width='40%'))
        
        
        self.main_ui = widgets.HBox([self.text_ui,self.action_ui])
        display(self.main_ui)

In [52]:
articles = make_articles('../data/interim/article_dicts/')

In [53]:
import pandas as pd          

selection = []
for a in articles:
    for section in a.sections:
        if "manikin" in section['text']:
            article = a.path.split('/')[-1]
            is_intro = 'introduction' in section['heading'].lower()
            is_dis = 'discussion' in section['heading'].lower()
            if not (is_intro or is_dis):
                selection.append(a)
selection = list(set(selection))

In [54]:
#articles = [a for a in articles if (a.annotations['split-half']==True)]

ad = Article_Displayer(selection, 
                       checks = ['AAT','AAT_uncertain','split-half','retest','reliability_uncertain','animal','drugs','food','general','manikin'],
                      #search = r'\brelia\w+',
                       search = 'manikin',
                      only_unreviewed = False)
# Todo: add manickin, joystick etc. to categorize AATs